In [2]:
import re
import time
import datetime
import requests
import pandas as pd
import multiprocessing
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from concurrent.futures import ThreadPoolExecutor
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException

from Script.Function.funciones import movies_for_category, categories, print_process, movies, series_informacion, desempaquetado, generador_dummies

# -------------- Parametros --------------
ruta_base = r"Bases_datos/"

# ----------------------------------------

In [2]:
print_process("Comienzo del Notebook 'Scraping'")

|-----------------------Comienzo del Notebook 'Scraping'-----------------------|


In [3]:
categorias = categories()

|---------------------Categorias capturadas correctamente.---------------------|
|-----------------------Categorias demoro 27.2 segundos-----------------------|


In [4]:

# Obtener la cantidad de núcleos disponibles en el sistema
num_nucleos = multiprocessing.cpu_count()

# Crear un diccionario para almacenar los resultados
resultados_dict = {}

categorias_comienzo = time.time()

with ThreadPoolExecutor(max_workers=num_nucleos) as executor:
    # Ejecutar la función para cada categoría en paralelo
    resultados = list(executor.map(movies_for_category, categorias))
    
    # Asignar los resultados al diccionario
    for categoria, resultado in zip(categorias, resultados):
        resultados_dict[categoria] = resultado

peliculas_total = []

for x,i in resultados_dict.items():
    for e in i:
        peliculas_total.append(e)

peliculas_total = set(peliculas_total)

categorias_fin = time.time()

print_process(f"La extraccion total de las categorias tubo una demora de {round((categorias_fin-categorias_comienzo),2)} segundos")


|Todo el contenido de la categoria 'biography' fue capturado correctamente, demoro 26.86 segundos|
|Todo el contenido de la categoria 'history' fue capturado correctamente, demoro 27.4 segundos|
|Todo el contenido de la categoria 'adventure' fue capturado correctamente, demoro 28.65 segundos|
|Todo el contenido de la categoria 'family' fue capturado correctamente, demoro 29.04 segundos|
|Todo el contenido de la categoria 'crime' fue capturado correctamente, demoro 29.49 segundos|
|Todo el contenido de la categoria 'fantasy' fue capturado correctamente, demoro 29.84 segundos|
|Todo el contenido de la categoria 'animation' fue capturado correctamente, demoro 31.83 segundos|
|Todo el contenido de la categoria 'action' fue capturado correctamente, demoro 32.73 segundos|
|Todo el contenido de la categoria 'horror' fue capturado correctamente, demoro 42.46 segundos|
|Todo el contenido de la categoria 'musical' fue capturado correctamente, demoro 14.35 segundos|
|Todo el contenido de la categ

In [5]:
peliculas_total = [x for x in peliculas_total if "/content/" in x]

In [6]:
print_process("Comienzo del proceso de extraccion de informacion de cada iteracion")

|-----Comienzo del proceso de extraccion de informacion de cada iteracion-----|


In [7]:
# Crear un diccionario para almacenar los resultados
todo = {"titulo": [],"idioma" : [], "subtitulos" : [], "ano": [], "duracion_minutos": [], "categorias": [], "sinopsis": [], "tipo": [], "link": []}

comienzo_movies = time.time()

with ThreadPoolExecutor(max_workers=num_nucleos) as executor:
    # Ejecutar la función para cada categoría en paralelo
    resultados = list(executor.map(movies, peliculas_total))

    try:
        for resultado in resultados:
            for clave, valor in resultado.items():
                todo[clave].append(valor)
    except:
        print()

fin_movies = time.time()

print_process(f"El proceso de captura de la informacion de todo el contenido fue exsitoso, y demoro {round((fin_movies - comienzo_movies) / 60,2)} minutos")


Error al cargar la url https://filmzie.com/content/the-wedding en el intento 1: 404


In [ ]:
df = pd.DataFrame(todo)

In [ ]:

df.to_csv(f"{ruta_base}Peliculas_Series_Completo_Respaldo.csv" , index=False)

In [3]:
df = pd.read_csv(f"{ruta_base}Peliculas_Series_Completo_Respaldo.csv")

In [4]:
peliculas = df.loc[df["tipo"] == "Pelicula"].reset_index(drop=True)
series = df.loc[df["tipo"] == "Serie"].reset_index(drop=True)

In [5]:

peliculas["duracion_minutos"] = peliculas["duracion_minutos"].str.replace(" min","").astype(int)
peliculas.loc[peliculas["idioma"] == 'Non linguistic content', "idioma"] = "No contiene idioma"
peliculas["subtitulos"] = peliculas["subtitulos"].fillna("No contiene subtitulos")


In [6]:
peliculas.head()

,titulo,idioma,subtitulos,ano,duracion_minutos,categorias,sinopsis,tipo,link
0,Donald Brittain: Filmmaker,English,No contiene subtitulos,1995,94,"Documentary, Biography",Delve into the captivating world of a renowned...,Pelicula,https://filmzie.com/content/donald-brittain-fi...
1,Knightesses of the Zodiaque,French,English (embedded),2020,25,"Drama, Comedy, Short, European Production",Planetary alignment. Convergence of the strugg...,Pelicula,https://filmzie.com/content/knightesses-of-the...
2,Night at the Hotel,Spanish,"English, Portuguese",2019,71,"Horror, Documentary",Two paranormal investigators filmed their over...,Pelicula,https://filmzie.com/content/night-at-the-hotel...
3,The Frame,English,English,2014,127,"Sci-Fi, Drama","Alex, a meticulous cargo thief for a dangerous...",Pelicula,https://filmzie.com/content/the-frame-2014
4,Lejos de Casa Éxodo Venezolano,Spanish,English,2020,87,"Drama, History","Samuel, a young Venezuelan, emigrates from his...",Pelicula,https://filmzie.com/content/lejos-de-casa-exod...


In [7]:
peliculas.to_csv(f"{ruta_base}Peliculas.csv",index=False)

In [8]:
peliculas_dummies = generador_dummies(peliculas)

In [9]:
# Exportamos base de datos de Peliculas con dummies
peliculas_dummies.to_csv(f"{ruta_base}Peliculas_dummies.csv",index=False)

peliculas_dummies.head()

,titulo,idioma,subtitulos,ano,duracion_minutos,sinopsis,tipo,link,Crime,War,...,Short,Biography,Western,Animation,Fantasy,Romance,Family,Comedy,History,Action
0,Donald Brittain: Filmmaker,English,No contiene subtitulos,1995,94,Delve into the captivating world of a renowned...,Pelicula,https://filmzie.com/content/donald-brittain-fi...,0,0,...,0,1,0,0,0,0,0,0,0,0
1,Knightesses of the Zodiaque,French,English (embedded),2020,25,Planetary alignment. Convergence of the strugg...,Pelicula,https://filmzie.com/content/knightesses-of-the...,0,0,...,1,0,0,0,0,0,0,1,0,0
2,Night at the Hotel,Spanish,"English, Portuguese",2019,71,Two paranormal investigators filmed their over...,Pelicula,https://filmzie.com/content/night-at-the-hotel...,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Frame,English,English,2014,127,"Alex, a meticulous cargo thief for a dangerous...",Pelicula,https://filmzie.com/content/the-frame-2014,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Lejos de Casa Éxodo Venezolano,Spanish,English,2020,87,"Samuel, a young Venezuelan, emigrates from his...",Pelicula,https://filmzie.com/content/lejos-de-casa-exod...,0,0,...,0,0,0,0,0,0,0,0,1,0


In [10]:
series.drop(columns=['duracion_minutos'], inplace=True)
series

,titulo,idioma,subtitulos,ano,categorias,sinopsis,tipo,link
0,Before the Fame,English,English,2018,"Biography, Documentary","""Before the Fame"" takes us down memory lane to...",Serie,https://filmzie.com/content/before-the-fame-2018
1,QuaranTeam!,English,English,2020,Comedy,"At Bork and Hitchens Advertising Agency, a mot...",Serie,https://filmzie.com/content/quaranteam-2020
2,Rescue Warriors,English,English,2005,Documentary,Discover the rigorous journey of USAF Pararesc...,Serie,https://filmzie.com/content/rescue-warriors-2005
3,Guy in Dubai,English,English,2017,Reality TV,"Join Paris Norriss in the adventure series ""Gu...",Serie,https://filmzie.com/content/guy-in-dubai-2017
4,Celebrity Fish N Fire,English,English,2016,"Adventure, Reality TV",Fish N Fire brings the energy of a chef and a ...,Serie,https://filmzie.com/content/celebrity-fish-n-f...
...,...,...,...,...,...,...,...,...
94,Bommi & Friends,English,English,2016,"Animation, Family","""Bommi & Friends"" follows young Bommi, who mov...",Serie,https://filmzie.com/content/bommi--friends-2016
95,Slope Slap,English,English,2019,Reality TV,Slope Slap is a big event close ended reality ...,Serie,https://filmzie.com/content/slope-slap-2019
96,Bulge Bracket,English,English,2020,Comedy,An ensemble office dramedy centered around a y...,Serie,https://filmzie.com/content/bulge-bracket-2020
97,2nd Generation,English,English,2018,Drama,Identity crisis is nothing new to anyone excep...,Serie,https://filmzie.com/content/2nd-generation-2018


In [11]:
links_series = list(series["link"].values)
links_series[:10]

['https://filmzie.com/content/before-the-fame-2018',
 'https://filmzie.com/content/quaranteam-2020',
 'https://filmzie.com/content/rescue-warriors-2005',
 'https://filmzie.com/content/guy-in-dubai-2017',
 'https://filmzie.com/content/celebrity-fish-n-fire-2016',
 'https://filmzie.com/content/inner-worlds-the-series-2012',
 'https://filmzie.com/content/vegan-mashup-2012',
 'https://filmzie.com/content/world-war-two-in-hd-2010',
 'https://filmzie.com/content/galactic-blitz-2015',
 'https://filmzie.com/content/the-benza-2019']

In [12]:
procesar = links_series[:3]

# Obtener la cantidad de núcleos disponibles en el sistema
num_nucleos = multiprocessing.cpu_count()

# Crear un diccionario para almacenar los resultados
resultados_dict = {"titulo": [], "temporadas": [], "capitulos": [], "sinopsis" : [], "duracion_minutos" : []}
# diccionario = {"titulo" : alm_3[0], "temporada" : alm_3[1], "capitulos" : alm_3[2], "sinopsis" : alm_3[3], "duracion_minutos" : alm_3[4]}
series_comienzo = time.time()

with ThreadPoolExecutor(max_workers=num_nucleos) as executor:
    # Ejecutar la función para cada categoría en paralelo
    resultados_temporales = list(executor.map(series_informacion, procesar))

# Separar los resultados temporales en listas individuales
titulos, temporadas, capitulos, sinopsis, duracion_minutos = zip(*resultados_temporales)

# Agregar los resultados a los diccionarios
resultados_dict["titulo"].extend(titulos)
resultados_dict["temporadas"].extend(temporadas)
resultados_dict["capitulos"].extend(capitulos)
resultados_dict["sinopsis"].extend(sinopsis)
resultados_dict["duracion_minutos"].extend(duracion_minutos)

series_fin = time.time()

print_process(f"La extraccion total de informacion de cada serie tuvo una demora de {round((series_fin-series_comienzo)/ 60, 2) } minutos")

Tiempo total de Rescue Warriors: 19.97 segundos
Tiempo total de Before the Fame: 21.16 segundos
Tiempo total de QuaranTeam!: 21.25 segundos
|La extraccion total de informacion de cada serie tuvo una demora de 0.42 minutos|


In [13]:
df_2 = pd.DataFrame(desempaquetado(resultados_dict))
df_2.head()

,titulo,temporadas,capitulos,sinopsis,duracion_minutos
0,Before the Fame,Season 1,1. Aries Spears,Taking a walk down memory lane with Aries Spea...,21
1,Before the Fame,Season 1,2. BeeGees,"Taking a walk down memory lane with BeeGees, t...",21
2,Before the Fame,Season 1,3. Chuck Liddell,Taking a walk down memory lane with Chuck Lidd...,21
3,Before the Fame,Season 1,4. DJ Khaled,"Taking a walk down memory lane with DJ Khaled,...",21
4,Before the Fame,Season 1,5. Dwyane Wade,Taking a walk down memory lane with Dwyane Wad...,21


In [16]:
pd.read_csv("Bases_datos\series_completo.csv")

,titulo,idioma,subtitulos,ano,categorias,sinopsis_x,tipo,link,temporadas,capitulos,sinopsis_capitulo,duracion_minutos
0,Before the Fame,English,English,2018,"Biography, Documentary","""Before the Fame"" takes us down memory lane to...",Serie,https://filmzie.com/content/before-the-fame-2018,Season 1,1. Aries Spears,Taking a walk down memory lane with Aries Spea...,21
1,Before the Fame,English,English,2018,"Biography, Documentary","""Before the Fame"" takes us down memory lane to...",Serie,https://filmzie.com/content/before-the-fame-2018,Season 1,2. BeeGees,"Taking a walk down memory lane with BeeGees, t...",21
2,Before the Fame,English,English,2018,"Biography, Documentary","""Before the Fame"" takes us down memory lane to...",Serie,https://filmzie.com/content/before-the-fame-2018,Season 1,3. Chuck Liddell,Taking a walk down memory lane with Chuck Lidd...,21
3,Before the Fame,English,English,2018,"Biography, Documentary","""Before the Fame"" takes us down memory lane to...",Serie,https://filmzie.com/content/before-the-fame-2018,Season 1,4. DJ Khaled,"Taking a walk down memory lane with DJ Khaled,...",21
4,Before the Fame,English,English,2018,"Biography, Documentary","""Before the Fame"" takes us down memory lane to...",Serie,https://filmzie.com/content/before-the-fame-2018,Season 1,5. Dwyane Wade,Taking a walk down memory lane with Dwyane Wad...,21
...,...,...,...,...,...,...,...,...,...,...,...,...
1033,The Beech Boys,English,English,2019,Comedy,An egomaniac and his dimwitted best friend mak...,Serie,https://filmzie.com/content/the-beech-boys-2019,Season 2,4. Sloop John B-Day,The one where Trey + Ethan go to Todd's birthd...,10
1034,The Beech Boys,English,English,2019,Comedy,An egomaniac and his dimwitted best friend mak...,Serie,https://filmzie.com/content/the-beech-boys-2019,Season 2,5. Tool Timin',The one where Trey + Ethan meet Cedric's new p...,10
1035,The Beech Boys,English,English,2019,Comedy,An egomaniac and his dimwitted best friend mak...,Serie,https://filmzie.com/content/the-beech-boys-2019,Season 2,6. Add Some Music to Your Trey,The one where Trey + Ethan start to drift apar...,15
1036,The Beech Boys,English,English,2019,Comedy,An egomaniac and his dimwitted best friend mak...,Serie,https://filmzie.com/content/the-beech-boys-2019,Season 2,7. Til I Die-agnosis,The one where Trey + Ethan host a telethon (& ...,13


In [14]:
series_descripcion = pd.merge(series,df_2, on="titulo", how="inner")
series_descripcion.head()

,titulo,idioma,subtitulos,ano,categorias,sinopsis_x,tipo,link,temporadas,capitulos,sinopsis_y,duracion_minutos
0,Before the Fame,English,English,2018,"Biography, Documentary","""Before the Fame"" takes us down memory lane to...",Serie,https://filmzie.com/content/before-the-fame-2018,Season 1,1. Aries Spears,Taking a walk down memory lane with Aries Spea...,21
1,Before the Fame,English,English,2018,"Biography, Documentary","""Before the Fame"" takes us down memory lane to...",Serie,https://filmzie.com/content/before-the-fame-2018,Season 1,2. BeeGees,"Taking a walk down memory lane with BeeGees, t...",21
2,Before the Fame,English,English,2018,"Biography, Documentary","""Before the Fame"" takes us down memory lane to...",Serie,https://filmzie.com/content/before-the-fame-2018,Season 1,3. Chuck Liddell,Taking a walk down memory lane with Chuck Lidd...,21
3,Before the Fame,English,English,2018,"Biography, Documentary","""Before the Fame"" takes us down memory lane to...",Serie,https://filmzie.com/content/before-the-fame-2018,Season 1,4. DJ Khaled,"Taking a walk down memory lane with DJ Khaled,...",21
4,Before the Fame,English,English,2018,"Biography, Documentary","""Before the Fame"" takes us down memory lane to...",Serie,https://filmzie.com/content/before-the-fame-2018,Season 1,5. Dwyane Wade,Taking a walk down memory lane with Dwyane Wad...,21
